# ПРЕПРОЦЕССИНГ ДАННЫХ 

In [1]:
import numpy as np

import os
from os.path import  join

In [2]:
clean_data_path = "D://test_neurolab//train//clean"
#clean_data_path = "C://input//neurolab//train//train//clean"
noisy_data_path = "D://test_neurolab//train//noisy"
#noisy_data_path = "C://input//neurolab//train//train//noisy"

СЧИТЫВАЕМ ДАННЫЕ ИЗ ТРЕНИНГОВОГО СЕТА

In [3]:
clean_data = []
for folder in os.listdir(clean_data_path):
    for file in os.listdir(join(clean_data_path, folder)):
        data = np.load(join(clean_data_path, folder, file))
        clean_data.append(data)
        
noisy_data = []
for folder in os.listdir(noisy_data_path):
    for file in os.listdir(join(noisy_data_path, folder)):
        data= np.load(join(noisy_data_path, folder, file))
        noisy_data.append(data)

ДЛЯ УНИФИКАЦИИ РАЗМЕРНОСТИ ДАННЫХ БУДЕМ БРАТЬ ЧАСТИ СПЕКТРОГРАММ ОКНАМИ РАЗМЕРА 80*80 И РАЗВОРАЧИВАТЬ ИХ В ВЕКТОРА

ОКНА С НЕПОЛНЫМ КОЛИЧЕСВТОМ ДАННЫХ БУДЕМ ДОПОЛНЯТЬИДЕНИЧНЫМИ ДАННЫМИ МЕТОДОМ np.pad

ОКНА, ГДЕ КОЛ-ВО ЗАПОЛНЕННЫХ СТОЛБЦОВ МЕНЬШЕ 30 БУДЕМ ИГНОРИРОВАТЬ

In [7]:
def get_rect_windows(mel, debug = False):
    if (debug): print("mel size", mel.shape)
    windows = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            windows.append(sample)
            
            if (debug): print(sample.shape)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            windows.append(pad_matrix)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    return windows
ex_data = clean_data[0]
a = get_rect_windows(ex_data, debug = True)

mel size (966, 80)
current size 0 79
(6400,)
current size 80 159
(6400,)
current size 160 239
(6400,)
current size 240 319
(6400,)
current size 320 399
(6400,)
current size 400 479
(6400,)
current size 480 559
(6400,)
current size 560 639
(6400,)
current size 640 719
(6400,)
current size 720 799
(6400,)
current size 800 879
(6400,)
current size 880 959
(6400,)
not enough real data left 7


ПОЛЬЗУЯСЬ ЗАДАННОЙ ФУНКЦИЕЙ ФОРМУИРЕМ НОВЫЙ ДАТАСЕТ СОСТОЯЩИЙ ИЗ ПОЛУЧЕННЫХ ВЕКТОРОВ

In [ ]:
clean_data_wind = []
for mel in clean_data:
    new_windows = get_rect_windows(mel)
    clean_data_wind.extend(new_windows)
    
noisy_data_wind = []
for mel in noisy_data:
    new_windows = get_rect_windows(mel)
    noisy_data_wind.extend(new_windows)

ФОРМИРУЕМ ИТОГОВЫЙ ТРЕНИНГОВЫЙ ДАТАСЕТ И ЦЕЛЕВУЮ ПЕРЕМЕННУЮ

In [ ]:
x_train = clean_data_wind + noisy_data_wind

y_train_clean = [0 for _ in range(len(clean_data_wind))]
y_train_noisy = [1 for _ in range(len(noisy_data_wind))]
y_train = y_train_clean + y_train_noisy

# ОБУЧЕНИЕ МОДЕЛЕЙ

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pickle

In [ ]:
#ЗАПУЩЕНО РАНЕЕ. ЗДЕСЬ ПРОСТО ЗАГРУЖАЕМ ПРЕДОБУЧЕННУЮ МОДЕЛЬ
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

In [14]:
logreg = pickle.load(open('logreg.sav', 'rb'))

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
#ЗАПУЩЕНО РАНЕЕ. ЗДЕСЬ ПРОСТО ЗАГРУЖАЕМ ПРЕДОБУЧЕННУЮ МОДЕЛЬ
tree_model = DecisionTreeClassifier(max_depth=3)  
# Fit a decision tree
tree_model = tree_model.fit(x_train, y_train) 

In [15]:
tree_model = pickle.load(open('tree_model.sav', 'rb'))

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
#ЗАПУЩЕНО РАНЕЕ. ЗДЕСЬ ПРОСТО ЗАГРУЖАЕМ ПРЕДОБУЧЕННУЮ МОДЕЛЬ
rf = RandomForestClassifier()  
rf_model = rf.fit(x_train, y_train)

In [16]:
rf_model = pickle.load(open('rf.sav', 'rb'))

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# ОЦЕНКА РЕЗУЛЬТАТОВ

In [26]:
import pandas as pd

РЕАЛИЗУЕМ ФУНКЦИЮ, КОТОРАЯ БУДЕТ ПРИНИМАТЬ НА ВХОД СПЕКТРОГРАММУ, РАЗБИВАТЬ ЕЕ НА ОКНА ПО ТОЙ ЖЕ ЛОГИКЕ, ЧТО И ТРЕНИРОВОЧНЫЕ ДАННЫЕ, ДЕЛАТЬ ОЦЕНКУ ДЛЯ КАЖДОГ ОКНА И ВЫДАВАТЬ ОБЩУЮ ОЦЕНКУ ДЛЯ ВСЕЙ СПЕКТРОГРАММЫ

In [20]:
def predict_by_mel(mel, model,debug = False):
    if (debug): print("mel size", mel.shape)
    predictions = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            
            current_prediction = model.predict(sample.reshape(1, -1))
            
            predictions.append(current_prediction)
            
            if (debug): print(current_prediction)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            current_prediction = model.predict(pad_matrix.reshape(1, -1))   
            predictions.append(current_prediction)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    av_predictions = np.mean(predictions) 
    if (debug) : print(predictions)   
    if(av_predictions > 0.5):
        final_prediction = 1
    else:
        final_prediction = 0
         
    return final_prediction
ex_data = clean_data[1]
predict_by_mel(ex_data, logreg, debug = True)

mel size (658, 80)
current size 0 79
[0]
current size 80 159
[0]
current size 160 239
[0]
current size 240 319
[0]
current size 320 399
[0]
current size 400 479
[0]
current size 480 559
[0]
current size 560 639
[0]
not enough real data left 19
[array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]


0

СЧИТЫВАЕМ ВАЛИДАЦИОННЫЕ ДАННЫЕ

In [18]:
#clean_data_path_val = "C://input//neurolab//val//clean"
clean_data_path_val = "D://test_neurolab//val//val//clean"

clean_data_val = []
for folder in os.listdir(clean_data_path_val):
    for file in os.listdir(join(clean_data_path_val, folder)):
        data = np.load(join(clean_data_path_val, folder, file))
        clean_data_val.append(data)
        
#noisy_data_path_val = "C://input//neurolab//val//noisy"
noisy_data_path_val = "D://test_neurolab//val//val//noisy"
noisy_data_val = []
for folder in os.listdir(noisy_data_path_val):
    for file in os.listdir(join(noisy_data_path_val, folder)):
        data= np.load(join(noisy_data_path_val, folder, file))
        noisy_data_val.append(data)

In [19]:
x_test = clean_data_val + noisy_data_val
y_train_clean_val = [0 for _ in range(len(clean_data_val))]
y_train_noisy_val = [1 for _ in range(len(noisy_data_val))]
y_test = y_train_clean_val + y_train_noisy_val

In [22]:
#Оцениваем Логрегрессию
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], logreg)
    predicted.append(pred)
    
logit_accuracy = metrics.accuracy_score(y_test, predicted)  
logit_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
logit_classification_report = metrics.classification_report(y_test, predicted)  
logit_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
logit_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
logit_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

In [23]:
#Оцениваем ДЕРЕВО РЕШЕНИЙ
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], tree_model)
    predicted.append(pred)
    
tree_accuracy = metrics.accuracy_score(y_test, predicted)  
tree_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
tree_classification_report = metrics.classification_report(y_test, predicted)  
tree_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
tree_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
tree_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

In [24]:
#Оцениваем СЛУЧАЙНЫЙ ЛЕС
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], rf_model)
    predicted.append(pred)
    
rf_accuracy = metrics.accuracy_score(y_test, predicted)  
rf_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
rf_classification_report = metrics.classification_report(y_test, predicted)  
rf_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
rf_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
rf_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

In [27]:
models = pd.DataFrame({  
  'Model': ['Logistic', 'd.Tree', 'r.f.'],
  'Accuracy' : [logit_accuracy, tree_accuracy, rf_accuracy],
  'Precision': [logit_precision, tree_precision, rf_precision],
  'recall' : [logit_recall, tree_recall, rf_recall],
  'F1' : [logit_f1, tree_f1, rf_f1]
  #'cv_precision' : [logit_cv_mean, tree_cv_mean, rf_cv_mean, svm_cv_mean, knn_cv_mean, bayes_cv_mean]
})
# Print table and sort by test precision
models.sort_values(by='Precision', ascending=False)

,Accuracy,F1,Model,Precision,recall
2,0.92700,0.921800,r.f.,0.992503,0.8605
1,0.87700,0.864463,d.Tree,0.962577,0.7845
0,0.90125,0.900126,Logistic,0.910486,0.8900


# ВЫВОД